In [1]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [2]:
! pip install langchain langchain-community langchain-openai promptquality openai pinecone-client --upgrade

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import random

# Prepare questions for the conversation
questions = pd.read_csv("../eval_questions.csv")["questions"].tolist()

In [3]:
import promptquality as pq

pq.login("console.staging.rungalileo.io")

/Users/quique/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


👋 You have logged into 🔭 Galileo (https://console.staging.rungalileo.io/) as pratik@rungalileo.io.


Config(console_url=Url('https://console.staging.rungalileo.io/'), username='pratik@rungalileo.io', password=SecretStr('**********'), token=SecretStr('**********'), current_user='pratik@rungalileo.io', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, api_url=Url('https://api.staging.rungalileo.io/'))

In [5]:
# VARIABLES WE WILL BE EXPERIMENTING WITH TODAY
CHUNK_SIZE = 400
EMBEDDING_MODEL = "text-embedding-3-small"
TOP_K = 4
CHAT_MODEL = "gpt-3.5-turbo-0125" # Alternative: "gpt-3.5-turbo-1106"

# Index Name
index_name = f"beauty-rc-cs{CHUNK_SIZE}-{EMBEDDING_MODEL}"

In [6]:
def create_tags():
    return [
        pq.RunTag(key="VectorDB Index", value=index_name, tag_type=pq.TagType.RAG),
        pq.RunTag(key="Encoder", value=EMBEDDING_MODEL, tag_type=pq.TagType.RAG),
        pq.RunTag(key="LLM", value=CHAT_MODEL, tag_type=pq.TagType.RAG),
        pq.RunTag(key="Top k", value=str(TOP_K), tag_type=pq.TagType.RAG)
    ]

In [7]:
from promptquality import Scorers

metrics = [
    # RAG Quality Metrics
    Scorers.context_adherence,
    Scorers.completeness_gpt,
    Scorers.chunk_attribution_utilization_gpt,
    # Other metrics I care about
    Scorers.latency,
    Scorers.pii,
    Scorers.toxicity,
    Scorers.tone,
]

In [8]:
from qa_chain import get_qa_chain
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
qa = get_qa_chain(embeddings, index_name, TOP_K, CHAT_MODEL, 0.1)


/Users/quique/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/Users/quique/Library/Python/3.9/lib/python/site-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [9]:
# Experiment Project and Run Names
project_name = "feb13-qa-pinecone"
run_name = f"{index_name}-{CHAT_MODEL}-k{TOP_K}"

In [10]:
from tqdm import tqdm

galileo_callback = pq.GalileoPromptCallback(project_name=project_name, 
                                            run_name=run_name, 
                                            scorers=metrics, 
                                            run_tags=create_tags())

qa.batch(questions, config=dict(callbacks=[galileo_callback]))
galileo_callback.finish()

Processing complete!: 100%|██████████| 5/5 [00:02<00:00,  2.41it/s]

🔭 View your prompt run on the Galileo console at: https://console.staging.rungalileo.io/prompt/chains/2428cf3a-3ceb-4fa5-be19-e398e7981027/44a29565-ad96-4fdd-8c34-8d88aba39df1?taskType=12
